In [1]:
# Import Libraries
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import re
import pandas as pd

In [2]:
path = "C:\\Users\\91636\\OneDrive\\Desktop\\Regex ML\\Selenium\\chromedriver.exe" #set driver path 

In [3]:
s = Service(path)  #create driver service

In [4]:
driver = webdriver.Chrome(service = s)  #create Chrome driver object
url = "https://www.google.com"
driver.get(url)

In [5]:
box = driver.find_element(By.XPATH, '//*[@id="APjFqb"]')  #find search box
box.send_keys("IMDB")  #enter IMDB in search box
box.send_keys(Keys.ENTER)  #hit enter
time.sleep(2)

In [6]:
driver.find_element(By.XPATH, '//*[@id="rso"]/div[1]/div/div/div/div/div/div/div/div[1]/div/span/a/h3').click()  #click first search result
time.sleep(2)

In [7]:
driver.find_element(By.XPATH, '//*[@id="imdbHeader-navDrawerOpen"]').click()
time.sleep(2)

In [8]:
driver.find_element(By.XPATH, '//*[@id="imdbHeader"]/div[2]/aside[1]/div/div[2]/div/div[1]/span/div/div/ul/a[3]/span').click()  #open side menu
time.sleep(2)

In [9]:
url = "https://www.imdb.com/chart/moviemeter/?ref_=nv_mv_mpm"

In [10]:
requests.get(url)  #GET request to url

<Response [403]>

In [11]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

In [12]:
source = requests.get(url, headers = headers)
source

<Response [200]>

In [13]:
soup = BeautifulSoup(source.content, 'html.parser')  #create soup
# soup

In [14]:
Rank = []
Movie_Name = []
Year = []
Rating = []
Votes = []

In [15]:
movie_data = soup.findAll('div', attrs = {'class':'ipc-metadata-list-summary-item__c'})  #find all movie divs

In [16]:
for i in movie_data:
    rank = i.find('div', class_='sc-b8b74125-0 eukXAN meter-const-ranking sc-be6f1408-6 dGVRjf cli-meter-title-header').text
    rank = re.sub(r'\([^)]*\)', '', rank)  #remove text in brackets
    Rank.append(rank)
    
    movie = i.a.h3.text
    Movie_Name.append(movie)
    
    value = i.find_all('span',class_ = "sc-be6f1408-8 fcCUPU cli-title-metadata-item")
    
    year = value[0].text
    Year.append(year)
    
    rating = i.find('span', class_= "sc-be6f1408-1 dbnleL").text.replace('Rate','')
    rating = re.sub(r'\([^)]*\)', '', rating)
    Rating.append(rating)
    
    vote = i.find('span', class_= "sc-be6f1408-1 dbnleL").text.replace('Rate','')
    vote = re.findall(r'\(([^)]*)\)', vote)  #extract text in brackets
    Votes.append(vote)

In [17]:
#create dataframe
df = pd.DataFrame({'Rank': Rank,'Name of movie': Movie_Name, 'Year of release': Year,'Rating': Rating, 'Votes': Votes})
df

,Rank,Name of movie,Year of release,Rating,Votes
0,1,Poor Things,2023,8.4,[79K]
1,2,Argylle,2024,6.0,[18K]
2,3,Saltburn,2023,7.1,[165K]
3,4,The Beekeeper,2024,6.4,[43K]
4,5,The Ministry of Ungentlemanly Warfare,2024,,[]
...,...,...,...,...,...
95,96,Queenpins,2021,6.4,[21K]
96,97,Fast X,2023,5.8,[114K]
97,98,The Family Plan,2023,6.3,[35K]
98,99,American Star,2024,5.3,[758]


In [18]:
df.to_csv("IMDB Rating.csv")  #save to csv